In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Deep").getOrCreate()

In [3]:
df = spark.read.csv("train.csv",inferSchema=True, header=True)

In [4]:
df.head(1)

[Row(Loan_ID='LP001002', Gender='Male', Married='No', Dependents='0', Education='Graduate', Self_Employed='No', ApplicantIncome=5849, CoapplicantIncome=0.0, LoanAmount=None, Loan_Amount_Term=360, Credit_History=1, Property_Area='Urban', Loan_Status='Y')]

In [5]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [6]:
df = df.drop("Loan_ID")

In [7]:
df.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [8]:
df = df.withColumn("Dependents",df.Dependents.cast("double"))

In [9]:
catList = [i[0] for i in df.dtypes if i[1] == 'string']
print(catList)

['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']


In [10]:
import pyspark.sql.functions as F

cols = [F.col(field[0]).cast('double') if field[1] == 'int' else F.col(field[0]) for field in df.dtypes]
df = df.select(cols)
df.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: double (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: double (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: double (nullable = true)
 |-- Loan_Amount_Term: double (nullable = true)
 |-- Credit_History: double (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [11]:
df = df.dropna()

In [12]:
numList = [i[0] for i in df.dtypes if i[1] == 'double']

In [13]:
catListIndexed = [i+'Indexed' for i in catList]
catListIndexed

['GenderIndexed',
 'MarriedIndexed',
 'EducationIndexed',
 'Self_EmployedIndexed',
 'Property_AreaIndexed',
 'Loan_StatusIndexed']

from pyspark.ml.feature import Imputer
imList = catList+numList
imoutList= [i+"imputer" for i in numList]
imputer = Imputer(inputCols=numList,outputCols=imoutList)

In [14]:
from pyspark.ml.feature import StringIndexer
catListIndexed = [i+'indexed' for i in catList]

stringindexer = StringIndexer(inputCols=catList,outputCols=catListIndexed)

In [15]:
finalCol = catListIndexed+numList
finalCol

['Genderindexed',
 'Marriedindexed',
 'Educationindexed',
 'Self_Employedindexed',
 'Property_Areaindexed',
 'Loan_Statusindexed',
 'Dependents',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [16]:
from pyspark.ml.feature import VectorAssembler
finalCol.remove('Loan_Statusindexed')
vectorAssembler = VectorAssembler(inputCols=finalCol,outputCol='features')

In [23]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',labelCol='Loan_Statusindexed')

In [24]:
from pyspark.ml import Pipeline
stage = [stringindexer,vectorAssembler,lr]
pipeline = Pipeline(stages=stage)

In [25]:
train,test = df.randomSplit([0.8,0.2],seed=42)

In [26]:
model = pipeline.fit(train)

In [27]:
pred = model.transform(test)

In [28]:
pred.select('prediction','Loan_Statusindexed').show()

+----------+------------------+
|prediction|Loan_Statusindexed|
+----------+------------------+
|       0.0|               0.0|
|       0.0|               1.0|
|       0.0|               0.0|
|       0.0|               0.0|
|       0.0|               0.0|
|       0.0|               0.0|
|       1.0|               1.0|
|       0.0|               1.0|
|       1.0|               1.0|
|       1.0|               0.0|
|       0.0|               0.0|
|       1.0|               0.0|
|       0.0|               0.0|
|       0.0|               0.0|
|       0.0|               0.0|
|       0.0|               1.0|
|       0.0|               0.0|
|       0.0|               0.0|
|       0.0|               1.0|
|       0.0|               1.0|
+----------+------------------+
only showing top 20 rows



In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create both evaluators
evaluator = BinaryClassificationEvaluator(labelCol="Loan_Statusindexed", rawPredictionCol="prediction")

'''acc = evaluatorMulti.evaluate(pred, {evaluatorMulti.metricName: "accuracy"})
f1 = evaluatorMulti.evaluate(pred, {evaluatorMulti.metricName: "f1"})
weightedPrecision = evaluatorMulti.evaluate(pred, {evaluatorMulti.metricName: "weightedPrecision"})
weightedRecall = evaluatorMulti.evaluate(pred, {evaluatorMulti.metricName: "weightedRecall"})'''
auc = evaluator.evaluate(pred)

In [30]:
auc

0.6458333333333333

In [31]:
acc = evaluator.evaluate(pred, {evaluator.metricName: 'areaUnderROC'})

In [32]:
auroc = evaluator.evaluate(pred, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(pred, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Area under ROC Curve: 0.6458
Area under PR Curve: 0.5556
